In [3]:
!git clone https://github.com/facebookresearch/pifuhd

Cloning into 'pifuhd'...
remote: Enumerating objects: 213, done.
remote: Total 213 (delta 0), reused 0 (delta 0), pack-reused 213
Receiving objects: 100% (213/213), 402.72 KiB | 7.74 MiB/s, done.
Resolving deltas: 100% (104/104), done.


In [5]:
cd /content/pifuhd/sample_images

/content/pifuhd/sample_images


In [ ]:
# Get frames from the input video

import cv2

vidcap = cv2.VideoCapture('/content/drive/MyDrive/JumpingJacks.mp4')
success,image = vidcap.read()
count = 0
while success:
  cv2.imwrite("frame%d.png" % count, image)     # save frame as JPEG file      
  success,image = vidcap.read()
  print('Read a new frame: ', success)
  count += 1

In [7]:
cd /content

/content


In [8]:
!git clone https://github.com/Daniil-Osokin/lightweight-human-pose-estimation.pytorch.git

Cloning into 'lightweight-human-pose-estimation.pytorch'...
remote: Enumerating objects: 120, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 120 (delta 1), reused 0 (delta 0), pack-reused 115
Receiving objects: 100% (120/120), 227.79 KiB | 4.75 MiB/s, done.
Resolving deltas: 100% (50/50), done.


In [9]:
cd /content/lightweight-human-pose-estimation.pytorch/

/content/lightweight-human-pose-estimation.pytorch


In [10]:
!wget https://download.01.org/opencv/openvino_training_extensions/models/human_pose_estimation/checkpoint_iter_370000.pth

--2022-01-17 17:06:16--  https://download.01.org/opencv/openvino_training_extensions/models/human_pose_estimation/checkpoint_iter_370000.pth
Resolving download.01.org (download.01.org)... 23.208.55.108, 2600:1402:3800:29b::4b21, 2600:1402:3800:2a3::4b21
Connecting to download.01.org (download.01.org)|23.208.55.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87959810 (84M) [application/octet-stream]
Saving to: ‘checkpoint_iter_370000.pth’

checkpoint_iter_370 100%[===================>]  83.88M  64.3MB/s    in 1.3s    

2022-01-17 17:06:18 (64.3 MB/s) - ‘checkpoint_iter_370000.pth’ saved [87959810/87959810]



In [11]:
#Function to get keypoints - from the authors of PIFuHD

import torch
import cv2
import numpy as np
from models.with_mobilenet import PoseEstimationWithMobileNet
from modules.keypoints import extract_keypoints, group_keypoints
from modules.load_state import load_state
from modules.pose import Pose, track_poses
import demo

def get_rect(net, images, height_size):
    net = net.eval()

    stride = 8
    upsample_ratio = 4
    num_keypoints = Pose.num_kpts
    previous_poses = []
    delay = 33
    for image in images:
        rect_path = image.replace('.%s' % (image.split('.')[-1]), '_rect.txt')
        img = cv2.imread(image, cv2.IMREAD_COLOR)
        orig_img = img.copy()
        orig_img = img.copy()
        heatmaps, pafs, scale, pad = demo.infer_fast(net, img, height_size, stride, upsample_ratio, cpu=False)

        total_keypoints_num = 0
        all_keypoints_by_type = []
        for kpt_idx in range(num_keypoints):  # 19th for bg
            total_keypoints_num += extract_keypoints(heatmaps[:, :, kpt_idx], all_keypoints_by_type, total_keypoints_num)

        pose_entries, all_keypoints = group_keypoints(all_keypoints_by_type, pafs)
        for kpt_id in range(all_keypoints.shape[0]):
            all_keypoints[kpt_id, 0] = (all_keypoints[kpt_id, 0] * stride / upsample_ratio - pad[1]) / scale
            all_keypoints[kpt_id, 1] = (all_keypoints[kpt_id, 1] * stride / upsample_ratio - pad[0]) / scale
        current_poses = []

        rects = []
        for n in range(len(pose_entries)):
            if len(pose_entries[n]) == 0:
                continue
            pose_keypoints = np.ones((num_keypoints, 2), dtype=np.int32) * -1
            valid_keypoints = []
            for kpt_id in range(num_keypoints):
                if pose_entries[n][kpt_id] != -1.0:  # keypoint was found
                    pose_keypoints[kpt_id, 0] = int(all_keypoints[int(pose_entries[n][kpt_id]), 0])
                    pose_keypoints[kpt_id, 1] = int(all_keypoints[int(pose_entries[n][kpt_id]), 1])
                    valid_keypoints.append([pose_keypoints[kpt_id, 0], pose_keypoints[kpt_id, 1]])
            valid_keypoints = np.array(valid_keypoints)
            
            if pose_entries[n][10] != -1.0 or pose_entries[n][13] != -1.0:
              pmin = valid_keypoints.min(0)
              pmax = valid_keypoints.max(0)

              center = (0.5 * (pmax[:2] + pmin[:2])).astype(np.int)
              radius = int(0.65 * max(pmax[0]-pmin[0], pmax[1]-pmin[1]))
            elif pose_entries[n][10] == -1.0 and pose_entries[n][13] == -1.0 and pose_entries[n][8] != -1.0 and pose_entries[n][11] != -1.0:
              # if leg is missing, use pelvis to get cropping
              center = (0.5 * (pose_keypoints[8] + pose_keypoints[11])).astype(np.int)
              radius = int(1.45*np.sqrt(((center[None,:] - valid_keypoints)**2).sum(1)).max(0))
              center[1] += int(0.05*radius)
            else:
              center = np.array([img.shape[1]//2,img.shape[0]//2])
              radius = max(img.shape[1]//2,img.shape[0]//2)

            x1 = center[0] - radius
            y1 = center[1] - radius

            rects.append([x1, y1, 2*radius, 2*radius])

        np.savetxt(rect_path, np.array(rects), fmt='%d')

In [ ]:
import os

img_dir = '/content/pifuhd/sample_images'

for file in os.listdir(img_dir):
  print(file)
  file_name = os.path.splitext(file)[0]

  # output paths
  obj_path = '/content/pifuhd/results/pifuhd_final/recon/result_%s_256.obj' % file_name
  out_img_path = '/content/pifuhd/results/pifuhd_final/recon/result_%s_256.png' % file_name
  video_path = '/content/pifuhd/results/pifuhd_final/recon/result_%s_256.mp4' % file_name
  video_display_path = '/content/pifuhd/results/pifuhd_final/result_%s_256_display.mp4' % file_name

  image_path = os.path.join(img_dir,file)
  
  if file.endswith('.png'):
    net = PoseEstimationWithMobileNet()
    checkpoint = torch.load('checkpoint_iter_370000.pth', map_location='cpu')
    load_state(net, checkpoint)

    get_rect(net.cuda(), [image_path], 512)


In [14]:
cd /content/pifuhd/

/content/pifuhd


In [15]:
!sh ./scripts/download_trained_model.sh

+ mkdir -p checkpoints
+ cd checkpoints
+ wget https://dl.fbaipublicfiles.com/pifuhd/checkpoints/pifuhd.pt pifuhd.pt
--2022-01-17 17:09:00--  https://dl.fbaipublicfiles.com/pifuhd/checkpoints/pifuhd.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 104.22.75.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1548375177 (1.4G) [application/octet-stream]
Saving to: ‘pifuhd.pt’

pifuhd.pt           100%[===================>]   1.44G  43.5MB/s    in 35s     

2022-01-17 17:09:36 (42.0 MB/s) - ‘pifuhd.pt’ saved [1548375177/1548375177]

--2022-01-17 17:09:36--  http://pifuhd.pt/
Resolving pifuhd.pt (pifuhd.pt)... failed: Name or service not known.
wget: unable to resolve host address ‘pifuhd.pt’
FINISHED --2022-01-17 17:09:36--
Total wall clock time: 36s
Downloaded: 1 files, 1.4G in 35s (42.0 MB/s)


In [16]:
#Inference with the pre-trained PIFuHD model
#COlab can only support meshes of resolution 256

!python -m apps.simple_test -r 256 --use_rect -i $img_dir

Resuming from  ./checkpoints/pifuhd.pt
test data size:  181
initialize network with normal
initialize network with normal
generate mesh (test) ...
  0% 0/181 [00:00<?, ?it/s]/content/drive/MyDrive/results/pifuhd_final/recon/result_frame0_256.obj
/content/pifuhd/lib/mesh_util.py:77: FutureWarning: marching_cubes_lewiner is deprecated in favor of marching_cubes. marching_cubes_lewiner will be removed in version 0.19
  verts, faces, normals, values = measure.marching_cubes_lewiner(sdf, thresh)
  1% 1/181 [00:05<16:39,  5.55s/it]/content/drive/MyDrive/results/pifuhd_final/recon/result_frame1_256.obj
  1% 2/181 [00:10<15:07,  5.07s/it]/content/drive/MyDrive/results/pifuhd_final/recon/result_frame10_256.obj
  2% 3/181 [00:15<14:34,  4.91s/it]/content/drive/MyDrive/results/pifuhd_final/recon/result_frame100_256.obj
  2% 4/181 [00:19<14:19,  4.86s/it]/content/drive/MyDrive/results/pifuhd_final/recon/result_frame101_256.obj
  3% 5/181 [00:24<14:09,  4.83s/it]/content/drive/MyDrive/results/pifuh